# IMPORT

In [1]:
import os
from pdf2image import convert_from_path
import ntpath
from google.oauth2 import service_account
from google.cloud import vision
import io

In [2]:
import pandas as pd

In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [5]:
from __future__ import unicode_literals, print_function
from spacy.lang.en import English # updated

# PDF TO TEXT

In [6]:
def convert_pdf_2_image(uploaded_image_path, uploaded_image):
    project_dir = os.getcwd()
    os.chdir(uploaded_image_path)
    file_name = str(uploaded_image).replace('.pdf','')
    pages = convert_from_path(uploaded_image, 200,poppler_path='/Users/kunal/Documents/VdartWorking/Poppler/poppler-0.68.0_x86/poppler-0.68.0/bin/')
    pageNumCount = 1
    outputNames = []
    for page in pages:
        output_file = file_name+"_"+str(pageNumCount) + '.jpg'
        page.save(output_file, 'JPEG')
        pageNumCount +=1
        outputNames.append(output_file)
    return outputNames

In [7]:
def convert_img_2_text(imagePath):
    keyDIR = "/Users/kunal/Documents/ResumeNLPVdart/APIKEYSGOOGLE/resumeMatcher-pdf2img.json"
    credentials = service_account.Credentials.from_service_account_file(keyDIR)
    client = vision.ImageAnnotatorClient(credentials=credentials)
    with io.open(imagePath, 'rb') as image_file:
        content = image_file.read()
    image = vision.types.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    totalString = ''
    for text in texts:
        totalString+=text.description
    totalString = totalString.rsplit(' ', 1)[0]
    return totalString

In [8]:
def deleteIMG(path):
    for i in os.listdir(path):
        if i.endswith(".jpg"):
            os.remove(folder_pdf + i)

In [83]:
folder_pdf = '/Users/kunal/Documents/ResumeNLPVdart/Testing_Delete/'
#deleteIMG(folder_pdf)

In [84]:
textList = []
numberPagesList = []
folder_pdf = '/Users/kunal/Documents/ResumeNLPVdart/Testing_Delete/'
for i in os.listdir(folder_pdf):
    if os.path.exists(folder_pdf + i[:-4] + ".jpg") == False:
        if i.endswith(".pdf"):
            imgName = convert_pdf_2_image(folder_pdf, i)
            print(imgName)
            print(len(imgName))
    for q in range(len(imgName)):
        text = convert_img_2_text(folder_pdf + i[:-4] + "_" + str(q+1) + ".jpg")
        textList.append(text)    
        numberPagesList.append(len(imgName))

['Document_402_1.jpg']
1
['Document_403_1.jpg']
1
['Document_405_1.jpg', 'Document_405_2.jpg', 'Document_405_3.jpg', 'Document_405_4.jpg', 'Document_405_5.jpg', 'Document_405_6.jpg']
6
['Document_406_1.jpg', 'Document_406_2.jpg', 'Document_406_3.jpg', 'Document_406_4.jpg', 'Document_406_5.jpg']
5
['Document_407_1.jpg', 'Document_407_2.jpg', 'Document_407_3.jpg']
3
['Document_408_1.jpg', 'Document_408_2.jpg']
2
['Document_409_1.jpg', 'Document_409_2.jpg']
2
['Document_410_1.jpg', 'Document_410_2.jpg', 'Document_410_3.jpg', 'Document_410_4.jpg', 'Document_410_5.jpg', 'Document_410_6.jpg', 'Document_410_7.jpg', 'Document_410_8.jpg', 'Document_410_9.jpg', 'Document_410_10.jpg', 'Document_410_11.jpg']
11
['Document_411_1.jpg']
1
['Document_412_1.jpg', 'Document_412_2.jpg']
2
['Document_413_1.jpg', 'Document_413_2.jpg']
2
['Document_414_1.jpg']
1
['Document_415_1.jpg']
1
['Document_417_1.jpg', 'Document_417_2.jpg']
2
['Document_418_1.jpg', 'Document_418_2.jpg', 'Document_418_3.jpg']
3
['Docu

## Convert to Excel

In [86]:
df = pd.DataFrame() 
df['Text Extracted'] = textList
df['DocumentNum'] = numberPagesList

In [87]:
os.chdir('/Users/kunal/Documents/ResumeNLPVdart/')

In [89]:
df.to_excel('alltextExtracted6.xlsx', index = False) 

## Read Excel

In [20]:
text = pd.ExcelFile("alltextExtracted6.xlsx")
dftext = text.parse("Sheet1")

In [21]:
dftext.head()

,Text Extracted,DocumentNum
0,"NISARGA HASSAN SREEDHAR\nSan Jose, California ...",1
1,"Rodolfo Cornel Jr.\n13214 Fidler Ave. Downey, ...",1
2,Microsoft\nCERTIFIED\naws certified\nProfessio...,6
3,Microsoft\nCERTIFIED\naws certified\nProfessio...,6
4,Microsoft\nCERTIFIED\naws certified\nProfessio...,6


In [22]:
text_list = dftext["Text Extracted"].tolist()
pagesList = dftext["DocumentNum"].tolist()

## Check for Recommendation letters

In [23]:
for i in text_list:
    if "Recommendation" in str(i): 
        print(i)
        print("")

EY
Tel: +1 212 773 3000
Ernst & Young LLP
5 Times Square
New York, NY 10036-6530
Fax: +1 212 773 6350
ey.com
Building a better
working world
April 30, 2018
Letter of Recommendation: Angelene Jean-Louis
Angelene and I were paired together as mentor-mentee for the Fall 2014 semester through
the Rutgers Business School (RBS) TeamUp mentoring program. The program provides RBS
students with the opportunity to have a one-to-one mentorship experience for a semester with
a business professional. The experience is intended to provide students, like Angelene, with
insights about the nature of the profession of their interest and guidance on how to be a
successful professional and future business leader.
From the moment I met her, I was encouraged and energized by Angelene's eagerness to
learn. We have kept in regular contact ever since, even after the one-semester program had
ended. Aside from spending time on her studies, she has continued to balance raising a family
and working full-time. Duri

In [127]:
CHECK FOR RECOMMENDATION

SyntaxError: invalid syntax (<ipython-input-127-ee05f2b841ec>, line 1)

In [24]:
totalText = []
for u in range(len(pagesList)):
    textExtracted = ""
    for h in range(pagesList[u]):
        try:
            textExtracted = textExtracted + "\n" + text_list[u]
        except:
            continue
        totalText.append(textExtracted)

In [25]:
totalText = []
for u in range(len(pagesList)):
    textExtracted = ""
    if pagesList[u] == 1:
        textExtracted = text_list[u]
        #print("Added " + text_list[u][:10])
    else:
        print(u)
        for h in range(pagesList[u]):
            if (u+h == len(text_list)):
                break
            if (isinstance(text_list[u+h], str) == False):
                continue
            textExtracted = textExtracted + "\n" + text_list[u+h]
        u += h
        print(u)
    totalText.append(textExtracted)

2
7
3
8
4
9
5
10
6
11
7
12
8
12
9
13
10
14
11
15
12
16
13
15
14
16
15
17
16
17
17
18
18
19
19
20
20
30
21
31
22
32
23
33
24
34
25
35
26
36
27
37
28
38
29
39
30
40
32
33
33
34
34
35
35
36
38
39
39
40
40
42
41
43
42
44
43
44
44
45
45
46
46
47
47
56
48
57
49
58
50
59
51
60
52
61
53
62
54
63
55
64
56
65
57
59
58
60
59
61
60
61
61
62
62
63
63
64
64
66
65
67
66
68
67
68
68
69
69
73
70
74
71
75
72
76
73
77
74
75
75
76
76
77
77
78
78
79
79
80


In [82]:
len(totalText)

80

In [ ]:
totalText[]

In [83]:
DOESNT WORK TO GET ALL PAGES TOGETHER

IndexError: list index out of range

## Var Names

In [11]:
print("Variable Name for text Files = ")
varNameL = []
for n, val in enumerate(textList):
    globals()["text%d"%n] = val
    varNameL.append("text%d"%n)
    #print("text%d"%n)
print("Total: " + str(n+1))

Variable Name for text Files = 
Total: 54


In [ ]:
textList

# Test SENTENCE SPLITTER

In [12]:
def splitSentences(totalText):
    raw_text = totalText
    nlp = English()
    nlp.add_pipe(nlp.create_pipe('sentencizer')) # updated
    doc = nlp(raw_text)
    sentences = [sent.string.strip() for sent in doc.sents]
    return sentences

In [13]:
def splitsentencespart2(totalText):
    sentences = [i for i in nlp(totalText).sents]
    return sentences

In [ ]:
totalSentence = []
for totalText in textList:
    arrayf = splitsentencespart2(totalText)
    for i in arrayf:
        totalSentence.append(i)

In [ ]:
len(totalSentence)

In [ ]:
df = pd.DataFrame() 
  
# Creating two columns 
df['Sentence'] = totalSentence

In [ ]:
import os
os.chdir('/Users/kunal/Documents/ResumeNLPVdart/')

In [ ]:
df.to_excel('resultsentences.xlsx', index = False) 

# TEST ENITITY EXTRACTION

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
def extract_emails(doc):
    resultlis = []
    for token in doc:
        if token.like_email:
            resultlis.append((token.text,token.idx, token.idx + len(token)))
    return resultlis
def extract_person_names(doc):
    personL = []
    for entity in doc.ents:
        if entity.label_=="PERSON":
            personL.append([entity.text, entity.start_char, entity.end_char])
    return personL

In [ ]:
print(textList[1])

In [ ]:
for i in textList:
    doc = nlp(i)
    EMAILLIST = extract_emails(doc)
    email_count = 1
    for email, start, end in EMAILLIST:
        print(str(email_count) + ":  " +  email)
        email_count+=1
    print("\n")
    NAMELIST = extract_person_names(doc)
    name_count = 1
    for name, start, end in NAMELIST:
        print(str(name_count) + ":  " + name)
        name_count +=1

# work 

In [ ]:
def get_hotwords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] # 1
    doc = nlp(text.lower()) # 2
    for token in doc:
        # 3
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        # 4
        if(token.pos_ in pos_tag):
            result.append(token.text)
                
    return result # 5

In [ ]:
tenserflow

In [ ]:
Hyperscince

In [ ]:
capture fast

# Create Sentences

In [15]:
setnecnceAuto = []
for line in textList[5].splitlines():
    setnecnceAuto.append(line)

In [ ]:
sentence = ""
sentenceList = []
for j in range(len(setnecnceAuto)):
    addword = input("Current: >>>"+ sentence+ "<<< + >>>"+ setnecnceAuto[j] + "<<<")
    if addword == "":
        sentence += setnecnceAuto[j] + " "
    elif addword == "g":
        sentenceList.append(sentence[:-1])
        print("Added >>>"+sentence[:-1]+"<<<")
        sentence = ""
        setnecnceAuto[j: j] = ["testfff"]
    elif addword == "fff":
        break

In [ ]:
word = ""
setnecnceAuto = []
for i in textList[5]:
    if not (i == "\n"):
        word += i 
    else:
        listofWords.append(word)
        word = ""

In [15]:
print(textList[5])

Microsoft
CERTIFIED
aws certified
Professional
Hem Chandra
Rockville, Maryland
469-436-9015
manoj@centillioninfotech.com
OBJECTIVE:
Lead Developer with more than 15+ years of experience in full-stack development (C#, VB.NET, .Net, ASP.NET,
ASP.NET MVC, Web API, Angular, AWS, SharePoint, InfoPath, SQL Server, SSIS, SSRS, Web services, WCF, LINQ,
XML, TFS 2010/13, SVN, VSS, GitHub, Telerik Controls, UML, JQuery and JavaScript) for building various desktop
and web applications.
SUMMARY:
Working experience on ASP.NET, ASP.NET MVC framework, Web API and Entity Framework.
Working experience on SharePoint 2013 and SharePoint 2010.
Proficiently learned AWS (Design, Deployment, AWS EC2, S3, Cloud watch, Load balancing & Lambda etc.).
Understanding of Microsoft Azure Cloud (Design, Deployment, app services, Azure Functions etc.).
Extensive experience in coding using C# and VB.NET
Implementation experience of various designs patterns (GOF).
Involved in Creation, Development and Deployment of SSIS

In [18]:
word = ""
listofWords = []
for i in textList[5]:
    if not (i == " " or i == "\n"):
        word += i 
    else:
        listofWords.append(word)
        word = ""

In [19]:
sentence = ""
sentenceList = []
for j in range(len(listofWords)):
    #print(listofWords[j])
    #print("running " + listofWords[j])
    addword = input("Current: >>>"+ sentence+ "<<< + >>>"+ listofWords[j] + "<<<")
    if addword == "":
        sentence += listofWords[j] + " "
    elif addword == "g":
        sentenceList.append(sentence[:-1])
        print("Added >>>"+sentence[:-1]+"<<<")
        sentence = ""
        listofWords[j: j] = ["testfff"]
    elif addword == "fff":
        break


Current: >>><<< + >>>Microsoft<<<
Current: >>>Microsoft <<< + >>>CERTIFIED<<<
Current: >>>Microsoft CERTIFIED <<< + >>>aws<<<g
Added >>>Microsoft CERTIFIED<<<
Current: >>><<< + >>>aws<<<
Current: >>>aws <<< + >>>certified<<<
Current: >>>aws certified <<< + >>>Professional<<<g
Added >>>aws certified<<<
Current: >>><<< + >>>Professional<<<
Current: >>>Professional <<< + >>>Hem<<<g
Added >>>Professional<<<
Current: >>><<< + >>>Hem<<<
Current: >>>Hem <<< + >>>Chandra<<<
Current: >>>Hem Chandra <<< + >>>Rockville,<<<g
Added >>>Hem Chandra<<<
Current: >>><<< + >>>Rockville,<<<
Current: >>>Rockville, <<< + >>>Maryland<<<
Current: >>>Rockville, Maryland <<< + >>>469-436-9015<<<g
Added >>>Rockville, Maryland<<<
Current: >>><<< + >>>469-436-9015<<<
Current: >>>469-436-9015 <<< + >>>manoj@centillioninfotech.com<<<g
Added >>>469-436-9015<<<
Current: >>><<< + >>>manoj@centillioninfotech.com<<<
Current: >>>manoj@centillioninfotech.com <<< + >>>OBJECTIVE:<<<g
Added >>>manoj@centillioninfotech.com<<<


Current: >>>OBJECTIVE: Lead Developer with more than 15+ years of experience in full-stack development (C#, VB.NET, .Net, ASP.NET, ASP.NET MVC, Web API, Angular, AWS, SharePoint, InfoPath, SQL Server, SSIS, SSRS, Web services, WCF, LINQ, XML, TFS 2010/13, SVN, VSS, GitHub, Telerik Controls, UML, JQuery <<< + >>>and<<<
Current: >>>OBJECTIVE: Lead Developer with more than 15+ years of experience in full-stack development (C#, VB.NET, .Net, ASP.NET, ASP.NET MVC, Web API, Angular, AWS, SharePoint, InfoPath, SQL Server, SSIS, SSRS, Web services, WCF, LINQ, XML, TFS 2010/13, SVN, VSS, GitHub, Telerik Controls, UML, JQuery and <<< + >>>JavaScript)<<<
Current: >>>OBJECTIVE: Lead Developer with more than 15+ years of experience in full-stack development (C#, VB.NET, .Net, ASP.NET, ASP.NET MVC, Web API, Angular, AWS, SharePoint, InfoPath, SQL Server, SSIS, SSRS, Web services, WCF, LINQ, XML, TFS 2010/13, SVN, VSS, GitHub, Telerik Controls, UML, JQuery and JavaScript) <<< + >>>for<<<
Current: >>>

Added >>>Understanding of Microsoft Azure Cloud (Design, Deployment, app services, Azure Functions etc.).<<<
Current: >>><<< + >>>Extensive<<<
Current: >>>Extensive <<< + >>>experience<<<
Current: >>>Extensive experience <<< + >>>in<<<
Current: >>>Extensive experience in <<< + >>>coding<<<
Current: >>>Extensive experience in coding <<< + >>>using<<<
Current: >>>Extensive experience in coding using <<< + >>>C#<<<
Current: >>>Extensive experience in coding using C# <<< + >>>and<<<
Current: >>>Extensive experience in coding using C# and <<< + >>>VB.NET<<<
Current: >>>Extensive experience in coding using C# and VB.NET <<< + >>>Implementation<<<h
Current: >>>Extensive experience in coding using C# and VB.NET <<< + >>>experience<<<
Current: >>>Extensive experience in coding using C# and VB.NET experience <<< + >>>of<<<
Current: >>>Extensive experience in coding using C# and VB.NET experience of <<< + >>>various<<<
Current: >>>Extensive experience in coding using C# and VB.NET experience of v

Current: >>>Extensive experience in developing UML Models using Visio and Rational <<< + >>>Rose.<<<
Current: >>>Extensive experience in developing UML Models using Visio and Rational Rose. <<< + >>>Practical<<<g
Added >>>Extensive experience in developing UML Models using Visio and Rational Rose.<<<
Current: >>><<< + >>>Practical<<<
Current: >>>Practical <<< + >>>experience<<<
Current: >>>Practical experience <<< + >>>to<<<
Current: >>>Practical experience to <<< + >>>implement<<<
Current: >>>Practical experience to implement <<< + >>>SOA<<<
Current: >>>Practical experience to implement SOA <<< + >>>(Service<<<
Current: >>>Practical experience to implement SOA (Service <<< + >>>Oriented<<<
Current: >>>Practical experience to implement SOA (Service Oriented <<< + >>>Architecture:<<<
Current: >>>Practical experience to implement SOA (Service Oriented Architecture: <<< + >>>WCF)<<<
Current: >>>Practical experience to implement SOA (Service Oriented Architecture: WCF) <<< + >>>and<<<
Curr

Current: >>>Foreclosure System-The Foreclosure Registration System is a web-based application for submission of those foreclosure-related notices and registrations that are <<< + >>>mandated<<<
Current: >>>Foreclosure System-The Foreclosure Registration System is a web-based application for submission of those foreclosure-related notices and registrations that are mandated <<< + >>>by<<<
Current: >>>Foreclosure System-The Foreclosure Registration System is a web-based application for submission of those foreclosure-related notices and registrations that are mandated by <<< + >>>Maryland<<<
Current: >>>Foreclosure System-The Foreclosure Registration System is a web-based application for submission of those foreclosure-related notices and registrations that are mandated by Maryland <<< + >>>law.<<<
Current: >>>Foreclosure System-The Foreclosure Registration System is a web-based application for submission of those foreclosure-related notices and registrations that are mandated by Marylan

Current: >>>NOI (Notice of Intent to Foreclose): Lender companies can submit NOI to DLLR. Notice is created in pdf and send to borrower thorough email as well as by post. User <<< + >>>can<<<
Current: >>>NOI (Notice of Intent to Foreclose): Lender companies can submit NOI to DLLR. Notice is created in pdf and send to borrower thorough email as well as by post. User can <<< + >>>also<<<
Current: >>>NOI (Notice of Intent to Foreclose): Lender companies can submit NOI to DLLR. Notice is created in pdf and send to borrower thorough email as well as by post. User can also <<< + >>>upload<<<
Current: >>>NOI (Notice of Intent to Foreclose): Lender companies can submit NOI to DLLR. Notice is created in pdf and send to borrower thorough email as well as by post. User can also upload <<< + >>>predefined<<<
Current: >>>NOI (Notice of Intent to Foreclose): Lender companies can submit NOI to DLLR. Notice is created in pdf and send to borrower thorough email as well as by post. User can also upload 

Current: >>>3. FPR (Foreclosed Property Registration): Lender company admin submits property registration of foreclosed property. Payment is done <<< + >>>by<<<
Current: >>>3. FPR (Foreclosed Property Registration): Lender company admin submits property registration of foreclosed property. Payment is done by <<< + >>>PayPal<<<
Current: >>>3. FPR (Foreclosed Property Registration): Lender company admin submits property registration of foreclosed property. Payment is done by PayPal <<< + >>>payment<<<


In [20]:
sentenceList

['Microsoft CERTIFIED',
 'aws certified',
 'Professional',
 'Hem Chandra',
 'Rockville, Maryland',
 '469-436-9015',
 'manoj@centillioninfotech.com',
 'OBJECTIVE: Lead Developer with more than 15+ years of experience in full-stack development (C#, VB.NET, .Net, ASP.NET, ASP.NET MVC, Web API, Angular, AWS, SharePoint, InfoPath, SQL Server, SSIS, SSRS, Web services, WCF, LINQ, XML, TFS 2010/13, SVN, VSS, GitHub, Telerik Controls, UML, JQuery and JavaScript) for building various desktop and web applications.',
 'SUMMARY:',
 'Working experience on ASP.NET, ASP.NET MVC framework, Web API and Entity Framework.',
 'Working experience on SharePoint 2013 and SharePoint 2010.',
 'Proficiently learned AWS (Design, Deployment, AWS EC2, S3, Cloud watch, Load balancing & Lambda etc.).',
 'Understanding of Microsoft Azure Cloud (Design, Deployment, app services, Azure Functions etc.).',
 'Extensive experience in coding using C# and VB.NET experience of various designs patterns (GOF).',
 'in Creation, 

In [ ]:
"""sentenceList.append("Tested and troubleshot configurations in the console to check the communication between the networks.")
sentenceList.append("Design of X-Band 8PSK Modulator using ADS")
sentenceList.append("Jan 2017 - April 2017")
sentenceList.append("Designed various components of the modulator used in a satellite at ISRO (Indian Space Research Organization), Bangalore.")
sentenceList.append("Performed optimization of the components at 8.75GHz frequency using the tools available in ADS to obtain the desired results of Insertion loss, Return loss and Isolation loss.")"""

In [ ]:
sentenceList

In [ ]:
len(sentenceList)

## FINAL SENTENCE LIST MADE

In [1]:
sentenceList = ['Microsoft CERTIFIED',
 'aws certified',
 'Professional',
 'Hem Chandra',
 'Rockville, Maryland',
 '469-436-9015',
 'manoj@centillioninfotech.com',
 'OBJECTIVE: Lead Developer with more than 15+ years of experience in full-stack development (C#, VB.NET, .Net, ASP.NET, ASP.NET MVC, Web API, Angular, AWS, SharePoint, InfoPath, SQL Server, SSIS, SSRS, Web services, WCF, LINQ, XML, TFS 2010/13, SVN, VSS, GitHub, Telerik Controls, UML, JQuery and JavaScript) for building various desktop and web applications.',
 'SUMMARY:',
 'Working experience on ASP.NET, ASP.NET MVC framework, Web API and Entity Framework.',
 'Working experience on SharePoint 2013 and SharePoint 2010.',
 'Proficiently learned AWS (Design, Deployment, AWS EC2, S3, Cloud watch, Load balancing & Lambda etc.).',
 'Understanding of Microsoft Azure Cloud (Design, Deployment, app services, Azure Functions etc.).',
 'Extensive experience in coding using C# and VB.NET experience of various designs patterns (GOF).',
 'in Creation, Development and Deployment of SSIS packages in SQL Server.',
 'Used SQL Profiler for Performance monitor to resolve Dead Locks and long running queries by checking appropriate changes to Transaction Isolation levels',
 'Database Performance of Index tuning with using Database Engine Tuning Advisor to resolve Performance issues.',
 'Performed and fine-tuned Stored Procedures, SQL Queries and User Defined Functions.',
 'Extensive experience in developing UML Models using Visio and Rational Rose.',
 'Practical experience to implement SOA (Service Oriented Architecture: WCF) and Three-tier architecture.',
 'Methodology: AGILE, Scrum, Test Driven Development.',
 'Worked as TFS administrator (TFS Server setup, installation, collection creation, build automation, branching & merging etc.)',
 'Vertical/Domain Experience: Travel, Banking, Insurance, Chemical, Water and Energy',
 'Proficiently learned new technology Angular to meet client need.',
 'PROFESSIONAL EXPERIENCE Maryland Department of Labor',
 'Lead Developer',
 'Description:',
 'Foreclosure System-The Foreclosure Registration System is a web-based application for submission of those foreclosure-related notices and registrations that are mandated by Maryland law. System verifies lender companies using Nationwide Multistate Licensing System & Registry',
 '(NMLS). Application is divided into three sub-systems. 1.',
 'NOI (Notice of Intent to Foreclose): Lender companies can submit NOI to DLLR. Notice is created in pdf and send to borrower thorough email as well as by post. User can also upload predefined csv files for bulk submission.',
 'SSIS package reads and saves data into database. System sends processed NOI status email to user.',
 '2. NOF (Notice of Foreclosure Filing): Lender submits NOF in the system which creates notice in pdf and sends NOF to borrower through email.']

In [5]:
df = pd.DataFrame(sentenceList)
df.to_excel("sentenceLISTNEW.xlsx")

In [3]:
import pandas as pd

# Create JSON from Array -- NOTWORKING

In [13]:
attributeCount = 3
sentenceCount = 2
text = "NISARGA HASSAN SREEDHAR"
attributes = [["NAME", "NISARGA HASSAN SREEDHAR", 0, 23],
              ["LOCATION", "San Jose, California", 12, 18],
              ["PROGRAMMING LANGUAGE", "PYTHON", 17, 20]]

In [ ]:
def createJSON(attributeCount, sentenceCount, text, attributes):
    totalString = """
    "sentenceCode": [
        "text": "textCODE",
        "entities" : [
    attributeCODE
        ]
    ],
    """
    attributeString = """       "attributeCode" : {
               "Atribname": "AtribnameCODE",
               "textDetected": "textDetectedCODE",
               "sChar": sCharCODE,
               "eChar": eCharCODE
           }
    """
    sentenceStringReplacer = "sentence" + str(sentenceCount)
    totalString =  totalString.replace("sentenceCode", sentenceStringReplacer)
    totalString =  totalString.replace("textCODE", text)
    tempStringReplacer = ""
    for i in range(attributeCount):
        tempStringReplacer+=("atriCODE"+str(i+1)+"\n")
    totalString = totalString.replace("attributeCODE", tempStringReplacer)
    for i in range(attributeCount):
        attributeStringNEW = attributeString
        print(attributes[i])
        attributeStringNEW = attributeStringNEW.replace("AtribnameCODE", attributes[i][0]).replace("textDetectedCODE", attributes[i][1])
        attributeStringNEW = attributeStringNEW.replace("sCharCODE", str(attributes[i][2])).replace("eCharCODE", str(attributes[i][3]))
        totalString = totalString.replace(("atriCODE"+str(i+1)), attributeStringNEW)
    return totalString

In [ ]:
data = """
{
    "Document1Sentences" : [
        "sentence1": [
            "text": "NISARGA HASSAN SREEDHAR",
            "entities" : [
                "attribute1" : {
                    "Atribname": "NAME",
                    "textDetected": "NISARGA HASSAN SREEDHAR",
                    "sChar": 0,
                    "eChar": 23
                }
            ]
        ],
        "sentence2": [
            "text": "San Jose, California",
            "entities" : [
                "attribute1" : {
                    "Atribname": "LOCATION",
                    "textDetected": "San Jose, California",
                    "sChar": 0,
                    "eChar": 18
                }
            ]
        ],
        "sentence3": [
            "text": "Programming: Python, Java",
            "entities" : [
                "attribute1" : {
                    "Atribname": "Programming Language",
                    "textDetected": "Python",
                    "sChar": 13,
                    "eChar": 18
                },
                "attribute2" : {
                    "Atribname": "Programming Language",
                    "textDetected": "Java",
                    "sChar": 20,
                    "eChar": 24
                }
            ]
        ],
        "sentence4": [
            "text": "Aug 2019 - Dec 2019",
            "entities" : [
                "attribute1" : {
                    "Atribname": "DATE",
                    "textDetected": "Aug 2019",
                    "sChar": 0,
                    "eChar": 12
                },
                "attribute2" : {
                    "Atribname": "DATE",
                    "textDetected": "Dec 2019",
                    "sChar": 16,
                    "eChar": 20
                }
            ]
        ]
    ]
}
"""

# Check Attributes for each Sentence in Document

In [ ]:
for i in sentenceList:
    print(checkattributes(i))

In [ ]:
## SPACY 

In [ ]:
def checkattributes(text):
    doc = nlp(text)
    EMAILLIST = extract_emails(doc)
    NAMELIST = extract_person_names(doc)
    #print(NAMELIST)
    #print(EMAILLIST)
    if (NAMELIST == [] and not EMAILLIST == []) or (not EMAILLIST == [] and not NAMELIST == []):
        return ["EMAIL", EMAILLIST[0][0], EMAILLIST[0][1], EMAILLIST[0][2]]
    #elif EMAILLIST == [] and not NAMELIST == []:
        #return ["NAME", NAMELIST[0][0], NAMELIST[0][1], NAMELIST[0][2]]
    #elif NAMELIST == [] and EMAILLIST == []:
        #return ["N/A", "" , 0,  0]
    else:
        return ["N/A", "" , 0,  0]
def extract_emails(doc):
    resultlis = []
    for token in doc:
        if token.like_email:
            resultlis.append((token.text,token.idx, token.idx + len(token)))
    return resultlis
def extract_person_names(doc):
    personL = []
    for entity in doc.ents:
        if entity.label_=="PERSON":
            personL.append([entity.text, entity.start_char, entity.end_char])
    return personL

# Convert Array to JSON

## Extract Excel and Turn into a list

In [26]:
os.chdir('/Users/kunal/Documents/ResumeNLPVdart/')

In [27]:
text = pd.ExcelFile("sentenceLISTT.xlsx")
attributes = text.parse("Sheet1")

In [28]:
attributes

,Text,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,NISARGA HASSAN SREEDHAR,Name,All,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"San Jose, California",Location,All,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,|+1 (925) 789-8911|,Phone,(925) 789-8911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,nisarga.nishu20@gmail.com,email,All,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,|,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
58,Tested and troubleshot configurations in the c...,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,Design of X-Band 8PSK Modulator using ADS,Hardware,X-Band 8PSK Modulator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,Jan 2017 - April 2017,Date,All,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,Designed various components of the modulator u...,Company,ISRO (Indian Space Research Organization),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
RowList = attributes.values.tolist()

In [30]:
attributeList = [[i for i in row if isinstance(i,str)] for row in RowList]

## Create Final List

In [31]:
totalTextRunning = totalText[0]
attributesNumberList = []
for j in attributeList:
    runningAttributeNum = []
    if len(j) == 2:
        continue
    if j[2] == "All":
        startChar = totalTextRunning.find(j[0])
        endChar = startChar + len(j[0])
        runningAttributeNum.extend([j[1], startChar, endChar])
    else:
        #print(int((len(j)-1)/2))
        for w in range(int((len(j)-1)/2)):
            startChar = totalTextRunning.find(j[2+w])
            endChar = startChar + len(j[2+w])
            runningAttributeNum.extend([j[1+w], startChar, endChar])
        attributesNumberList.append(runningAttributeNum)

In [32]:
def sentence_2_word(sentence):
    word = ""
    listofWords = []
    for i in sentence:
        if not (i == " " or i == "\n"):
            word += i 
        else:
            listofWords.append(word)
            word = ""
    if len(listofWords) == 0:
        listofWords.append(sentence)
    return listofWords

In [33]:
def find_all_indexes(input_str, search_str):
    l1 = []
    length = len(input_str)
    index = 0
    while index < length:
        i = input_str.find(search_str, index)
        if i == -1:
            return l1
        l1.append(i)
        index = i + 1
    return l1[0]

In [34]:
totalTextRunning = totalText[0]
attributesNumberList = []
for j in attributeList:
    runningAttributeNum = []
    if len(j) == 2:
        continue
    if j[2] == "All":
        startChar = totalTextRunning.find(j[0])
        endChar = startChar + len(j[0])
        runningAttributeNum.extend([j[1].upper() , startChar, endChar])
        attributesNumberList.append(runningAttributeNum)
    else:
        #print("RUNNING " + str(int((len(j)-1)/2)))
        for w in range(0, int((len(j)-1)), 2):
            runningAttributeNum = []
            if not totalTextRunning.find(j[2+w]) == -1:
                startChar = totalTextRunning.find(j[2+w])
                endChar = startChar + len(j[2+w])
                runningAttributeNum.extend([j[1+w].upper(), startChar, endChar])
            else:
                words = sentence_2_word(j[2+w])
                detected = 0
                for g in words:
                    if not j[2+w].find(g) == -1:
                        #charTemp = j[2+w].find(g)
                        detected+=1
                if detected/len(words) > 0.95:
                    startChar = totalTextRunning.find(words[0])
                    endChar = startChar + len(words[0])
                    runningAttributeNum.extend([words[0].upper(), startChar, endChar])
                else:
                    charTemp = -1
                    print("NOT DETECTED")
                    endChar , startChar = -1
                    runningAttributeNum.extend(["NOTDETECTED", startChar, endChar])
            attributesNumberList.append(runningAttributeNum)

In [35]:
for testing in attributesNumberList:
    if testing[1] == -1:
        attributesNumberList.remove(testing)

## DataSet to JSON FILE

In [36]:
attributeCount = len(attributesNumberList)
text = totalTextRunning
attributes = attributesNumberList

In [38]:
def createJSONFILE(attributeCount, text, attributes):
    totalString = """
    {
        "annotations": [
            attributeCODE
        ],
        "text_snippet": {
            "content": "textCODE"
        }
    }

    """
    attributeString = """       
        {
          "text_extraction": {
            "text_segment": {
              "end_offset": eCharCODE,
              "start_offset": sCharCODE
            }
          },
          "display_name": "AtribnameCODE"
        },
    """
    totalString =  totalString.replace("textCODE", text)
    tempStringReplacer = ""
    for i in range(attributeCount):
        tempStringReplacer+=("atriCODE"+str(i+1)+"\n")
    totalString = totalString.replace("attributeCODE", tempStringReplacer)
    for i in range(attributeCount):
        attributeStringNEW = attributeString
        #print(attributes[i])
        attributeStringNEW = attributeStringNEW.replace("AtribnameCODE", attributes[i][0])
        attributeStringNEW = attributeStringNEW.replace("sCharCODE", str(attributes[i][1])).replace("eCharCODE", str(attributes[i][2]))
        totalString = totalString.replace(("atriCODE"+str(i+1)), attributeStringNEW)
    return totalString

## Final Run Create JSON

In [40]:
print(createJSONFILE(attributeCount, text, attributes))


    {
        "annotations": [
                   
        {
          "text_extraction": {
            "text_segment": {
              "end_offset": 23,
              "start_offset": 0
            }
          },
          "display_name": "NAME"
        },
    
       
        {
          "text_extraction": {
            "text_segment": {
              "end_offset": 44,
              "start_offset": 24
            }
          },
          "display_name": "LOCATION"
        },
    
       
        {
          "text_extraction": {
            "text_segment": {
              "end_offset": 63,
              "start_offset": 49
            }
          },
          "display_name": "PHONE"
        },
    
       
        {
          "text_extraction": {
            "text_segment": {
              "end_offset": 90,
              "start_offset": 65
            }
          },
          "display_name": "EMAIL"
        },
    
       
        {
          "text_extraction": {
            "text_segm

## Test DataSet

In [ ]:
dataset = """{
  "annotations": [
    {
      "text_extraction": {
        "text_segment": {
          "end_offset": 67,
          "start_offset": 62
        }
      },
      "display_name": "Modifier"
    },
    {
      "text_extraction": {
        "text_segment": {
          "end_offset": 158,
          "start_offset": 141
        }
      },
      "display_name": "SpecificDisease"
    }
  ],
  "text_snippet": {
    "content": "10051005\tA common MSH2 mutation in English and North American HNPCC families:
      origin, phenotypic expression, and sex specific differences in colorectal cancer .\tThe
      frequency , origin , and phenotypic expression of a germline MSH2 gene mutation previously
      identified in seven kindreds with hereditary non-polyposis cancer syndrome (HNPCC) was
      investigated . The mutation ( A-- > T at nt943 + 3 ) disrupts the 3 splice site of exon 5
      leading to the deletion of this exon from MSH2 mRNA and represents the only frequent MSH2
      mutation so far reported . Although this mutation was initially detected in four of 33
      colorectal cancer families analysed from eastern England , more extensive analysis has
      reduced the frequency to fou"
  }
}"""

# Test Creating Loop for creating Sentences 

In [ ]:
wordCount = 0
elementList = []
for i in listofWords:
    elementGoing = ""
    print(str(wordCount) +  "   " +  i)
    elementContain = input("Does this contain a element")
    while elementContain == "" or not (elementContain == "n" or elementContain == "y") :
        elementContain = input("Does this contain a element")
    if elementContain == "Y" or elementContain == "y":
        elementGoing += i
        continueaddElement = input("Add more to this element?")
        while continueaddElement == "":
            continueaddElement = input("Add more to this element?")
        while continueaddElement == "Y" or continueaddElement == "y":
            print(listofWords[wordCount+1])
            countineElement2  = input("Add more to this element?")
            while countineElement2 == "":
                countineElement2 = input("Add more to this element?")
            while countineElement2 == "Y" or countineElement2 == "y":
                print(listofWords[wordCount+2])
                countineElement2 =  input("Add more to this element?")
        if continueaddElement == "x":
            break
    if elementContain == "x":
        break
    wordCount+=1

0   NISARGA
Does this contain a elementy
Add more to this element?y
HASSAN
Add more to this element?y
SREEDHAR
San
Add more to this element?n
HASSAN
Add more to this element?n
HASSAN
Add more to this element?n
HASSAN
Add more to this element?q
HASSAN
Add more to this element?x
HASSAN
Add more to this element?x
HASSAN
Add more to this element?x
HASSAN


# Write a program that goes through the text file and extracts each entity and its starting position and ending position and its name